<a href="https://colab.research.google.com/github/cristianomg10/uniasselvi-projeto-ii/blob/main/Projeto_II_Futuro_dos_Dados_e_Aplica%C3%A7%C3%A3o_de_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto II - Futuro dos Dados e Aplicação de Big Data

**Estudante:** Cristiano M. Garcia (2874011)

**Curso:** Big Data e Inteligência Analítica

## Introdução e Fundamentação Teórica

>### Definição do Tema

A Análise de Sentimentos é uma atividade primordial para instituições que queiram acompanhar e entender melhor a opinião de seus clientes sobre produtos ou serviços oferecidos.

A Análise de Sentimentos permite o entendimento de emoções de clientes, suas necessidades, podendo gerar informações para melhoridas de produtos e de experiência do cliente com os serviços prestados (ou consumidos).



>### Objetivo

O objetivo deste projeto é a geração de um modelo de classificação de estrelas do conjunto de dados do **Yelp Dataset**. Em outras palavras, a ideia é gerar um classificador que seja capaz de, a partir de um *review* de usuário e outras características, categorizar o texto em uma das 5 categorias disponíveis, que correspondem a **número de estrelas, variando de 1 a 5**.

>### Especificação Técnica

>#### Conjunto de dados

O conjunto de dados foi obtido do site https://www.yelp.com/dataset. O conjunto de dados possui quase 7 milhões de *reviews*, proveniente de mais de 150 mil negócios nos EUA.

Este conjunto de dados possui diversas colunas, entre elas: 
* ***Review***, correspondendo à postagem;
* **Data do *review***, correspondendo à data de postagem no formato aaaa-mm-dd;
* **Número de estrelas**, variando de 1 a 5;
* **ID do negócio** correspondente ao *review*;
* **Estado** de localização do negócio, correspondendo aos estados dos EUA.

Como hipótese inicial, têm-se que o campos **review, data e número de estrelas** podem ser úteis. O review é o que será classificado, a data pode ajudar pois o padrão de escrita é modificado ao longo do tempo de forma não-intencional pelas pessoas, enquanto o número de estrelas será a categoria (classe). 

>#### Métricas

Os dados, considerando as classes, está desbalanceado, o que torna necessária a aplicação de métricas para considerar o desbalanceamento. Sendo assim, as métricas a serem utilizadas são as métricas:

* Acurácia; 
* F1-Score;
* Coeficiente de Correlação de Matthews, que varia de -1 a 1, onde abaixo de 0 significa graus de correlação negativa, e acima de 0, graus de correlação positiva entre o número real de estrelas e o número de estrelas predito pelo classificador;
* Cohen's Kappa, que mede o índice de concordância entre 2 avaliadores (número real de estrelas, e o número de estrelas predito pelo classificador).

>#### Classificadores

Considerando a quantidade de dados disponíveis nesse conjunto de dados, uma extração de características utilizando **Bag-of-Words** pode gerar dados muito esparsos e de alta dimensionalidade (característica natural de representação de textos). Dessa forma, é pretendido usar 2 classificadores: 

* Support Vector Machine (SVM), com Bag-of-Words. SVM é conhecido por sua habilidade de lidar bem com dados de alta dimensionalidade;
* Redes Neurais Artificiais, com Embeddings e usando LSTM.

>#### Pré-processamento

Como métodos de pré-processamento, será usado stemming. Para o SVM, serão removidas as stopwords.

>#### Bibliotecas

Pretende-se usar neste projeto as bibliotecas Sci-kit Learn e Keras.


>### Metodologia

A metodologia seguirá os seguintes passos, conforme alguns pontos já mencionados acima:

* Pré-processamento;
* Treinamento dos modelos;
* Avaliação seguindo os critérios definidos.

Para o SVM, será utilizado:
* Transformação para minúsculo;
* Remoção de stopwords e pontuação;
* Stemming;
* Uso de bag-of-words.

Para a Rede Neural, será utilizado:
* Transformação para minúsculo.

A remoção de stopwords não favorece os embeddings pois os embeddings são gerados a partir das palavras e associações com suas respectivas palavras vizinhas.
